<a href="https://colab.research.google.com/github/ProjectProRepo/How-to-Build-an-FAQ-ChatBot-using-Open-source-LLM-/blob/main/FAQ_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio transformers accelerate pypdf PyPDF2 bitsandbytes

In [ ]:
import gradio as gr
from PyPDF2 import PdfReader
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  # Login with your HF token

In [ ]:
# Load LLM
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    load_in_8bit=True
)

llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9
)


In [ ]:
# Extract and cache full PDF text (as list of chunks)
def extract_text_chunks(pdf_file, chunk_size=1500, overlap=200):
    reader = PdfReader(pdf_file)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() or ""

    chunks = []
    start = 0
    while start < len(full_text):
        end = start + chunk_size
        chunks.append(full_text[start:end])
        start += chunk_size - overlap
    return chunks

# Find best matching chunk based on query keywords
def find_relevant_chunk(chunks, query):
    best_score = 0
    best_chunk = ""
    query_words = set(query.lower().split())

    for chunk in chunks:
        chunk_words = set(chunk.lower().split())
        score = len(query_words.intersection(chunk_words))
        if score > best_score:
            best_score = score
            best_chunk = chunk
    return best_chunk

# Generate answer using LLM
def answer_query_from_pdf(pdf_file, query):
    chunks = extract_text_chunks(pdf_file)
    relevant_chunk = find_relevant_chunk(chunks, query)

    prompt = (
        f"You are a helpful assistant. Based on the following document excerpt:\n\n"
        f"{relevant_chunk}\n\n"
        f"Answer this question: {query}"
    )

    result = llm(prompt)[0]["generated_text"]
    return result.replace(prompt, "").strip()

# Gradio UI
iface = gr.Interface(
    fn=answer_query_from_pdf,
    inputs=[
        gr.File(file_types=[".pdf"], label="Upload a large PDF"),
        gr.Textbox(lines=2, placeholder="Ask a question about the PDF...", label="Your Question")
    ],
    outputs="text",
    title="🔍 Ask Questions from a Large PDF",
    description="Upload a large PDF (up to 22MB) and ask questions. The bot finds relevant text and answers using an open-source LLM."
)

if __name__ == "__main__":
    iface.launch()
